<a href="https://colab.research.google.com/github/GergelyTuri/sleep/blob/master/notebooks/sleep_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clustering sleep data
* first iteration by Fiona
* 5/2/2023 adding data loader GT and running some cleanup
* 5/2/2023 added to github

In [2]:
!git clone https://github.com/GergelyTuri/sleep.git

Cloning into 'sleep'...
remote: Enumerating objects: 211, done.
remote: Counting objects: 100% (211/211), done.
remote: Compressing objects: 100% (150/150), done.
remote: Total 211 (delta 107), reused 144 (delta 50), pack-reused 0
Receiving objects: 100% (211/211), 696.41 KiB | 20.48 MiB/s, done.
Resolving deltas: 100% (107/107), done.


#### run these two lines in case you need to update the codebase

In [1]:
%cd '/content/sleep/'
!git pull

[Errno 2] No such file or directory: '/content/sleep/'
/content
fatal: not a git repository (or any of the parent directories): .git


In [3]:
import sys
from pathlib import Path
# from os.path import join

import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
import seaborn as sns

sys.path.append('/content/sleep/')

from sleep.base import core
from sleep.base.xcorr import xcorr_analysis as xa

In [5]:
!pip install tslearn
from tslearn.clustering import KShape, TimeSeriesKMeans
from tslearn.utils import to_time_series_dataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Mounting Drive

In [3]:
drive = core.GoogleDrive()
core.mount_drive()

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


### Available mice

In [4]:
db = core.MouseDatabase()
mice = db.load_mouse_database()
mice

,mouseID,day,sessionID,cellType,trialID,eeg_data,significantCells,sima folder,note
1,5HT2afl05b_1,7_7_SD,TSeries-07072021-0833_GC1-001,GC,747,velo_eeg.csv,redo,TSeries-07072021-0833_GC1-001_Cycle00001_Eleme...,
2,5HT2afl05b_1,7_7_SD,TSeries-07072021-0833_GC2-003,GC,745,velo_eeg.csv,redo,TSeries-07072021-0833_GC2-003_Cycle00001_Eleme...,"looks good, uploaded"
3,5HT2afl05b_1,7_7_SD,TSeries-07072021-0833_MC1-002,MC,740,velo_eeg.csv,redo,TSeries-07072021-0833_MC1-002_Cycle00001_Eleme...,
4,5HT2afl05b_1,7_7_SD,TSeries-07072021-0833_MC2-004,MC,749,velo_eeg.csv,redo,TSeries-07072021-0833_MC2-004_Cycle00001_Eleme...,"nice, uploaded"
5,5HT2afl05b_2,7_6_SD,TSeries-07062021-0844_GC-001,GC,762,velo_eeg.csv,redo,TSeries-07062021-0844_GC-001_Cycle00001_Elemen...,uploaded
6,5HT2afl05b_2,7_6_SD,TSeries-07062021-0844_MC-002,MC,761,velo_eeg.csv,redo,TSeries-07062021-0844_MC-002_Cycle00001_Elemen...,uploaded
7,5HT2afl05b_2,7_6_SD,TSeries-07062021-0844_MC2-004,MC,760,velo_eeg.csv,redo,TSeries-07062021-0844_MC2-004_Cycle00001_Eleme...,"best, uploaded"
8,126031_5,11_23,TSeries-11232022-GC-MC-002,GC,876,velo_eeg.csv,redo,TSeries-11232022-GC-MC-002.sima,"very nice 2p, but ca signal craps off after ab..."


In [5]:
data_dir = '/gdrive/Shareddrives/Turi_lab/Data/Sleep/2p/Analysis/data/'

In [5]:
db.add_mouse_to_database(['foo', 'bar', 'goo'])

4

### Loading a set of data from one mouse. 
This line will return a dictionary with the data

In [10]:
data_dict = xa.data_loader(data_dir, 'dfof', mice.loc[1,'mouseID'],
               mice.loc[1,'day'],
               mice.loc[1,'sessionID'])
data_dict.keys()

number of cells in this recording: 94


dict_keys(['dfof', 'significant_cells', 'eeg_velocity'])

In [ ]:
data= pd.read_csv("/cell2.csv")
data.head()
data = data.fillna(data.mean())

In [ ]:
data.head()

,roi_label,dfof,sleep_numeric,cell_numeric,time
0,3210110,0.069911,1,1,1
1,3210110,0.623665,1,1,2
2,3210110,0.437703,1,1,3
3,3210110,0.773147,1,1,4
4,3210110,0.506952,1,1,5


In [ ]:
# Split the data into two clusters based on the 'cell' column
cluster_1 = data[data['cell_numeric'] == 1].drop('cell_numeric', axis=1).values
cluster_2 = data[data['cell_numeric'] == 0].drop('cell_numeric', axis=1).values

# Cluster each cluster separately
ks = KShape(n_clusters=2, n_init=10, verbose=1)
clustered_1 = ks.fit_predict(cluster_1)
clustered_2 = ks.fit_predict(cluster_2)

# Merge the clustered data back into one DataFrame
clustered_data = pd.DataFrame({'cell_1': clustered_1, 'cell_0': clustered_2})

Init 1


In [ ]:
# Calculate the percentage of upregulated timepoints for each cluster
upreg_pct = pd.DataFrame()
for cluster in clustered_data['cluster'].unique():
    cluster_data = clustered_data.loc[clustered_data['cluster']==cluster,:]
    upreg_pct.loc[cluster,'upreg_pct'] = np.mean(cluster_data['upregulated'])
    
# Plot the results
plt.bar(x=upreg_pct.index, height=upreg_pct['upreg_pct'])
plt.xlabel('Cluster')
plt.ylabel('% Upregulated')
plt.show()

In [ ]:
train =[[1,2,3,4],[2,3,4,5,6,10],[2,3,3,3,3,3],[4,5,5,5,6,9,10],[10,2,4,5,9,3,5],]
#status=[celltype,sleep/awake,roi_label]* number of time series

X = to_time_series_dataset(train)
 
#train=np.arange(25).reshape((5, 5))
 
model1 = TimeSeriesKMeans(n_clusters=3, metric="dtw",
                         max_iter=10, random_state=1)
 
model1.fit(X)
print(model1.fit_predict(X))
print(model1.predict(X))

In [ ]:
mydata= pd.read_csv("/content/cell2.csv")
mydata.head()
# Keep only the first 50000 rows
mydata = mydata.head(50000)
# Extract the dfof and sleep variables
dfof = mydata['dfof'].values
sleep = mydata['sleep_numeric'].values
roi_label = mydata['roi_label'].values

# Find the indices where the sleep value changes
sleep_changes = np.where(np.diff(sleep) != 0)[0] + 1

# Split the dfof array based on the sleep changes
dfof_periods = np.split(dfof, sleep_changes)


# Create a 2D array where each element is an array corresponding to a period between sleep value changes
dfof_matrix = np.empty((len(dfof_periods),), dtype=object)
dfof_matrix[:] = dfof_periods

# Print the shape of the matrix
print(dfof_matrix.shape)


(19,)


In [ ]:
dfof_matrix[2].shape

(11645,)

In [ ]:
X = to_time_series_dataset(dfof_matrix)
 
model1 = TimeSeriesKMeans(n_clusters=10, metric="dtw",
                         max_iter=10, random_state=1)
 
model1.fit(X)
print(model1.fit_predict(X))
print(model1.predict(X))

In [ ]:
mydata.head()

,roi_label,dfof,sleep_numeric,cell_numeric,time
0,3210110,0.069911,1.0,1.0,1.0
1,3210110,0.623665,1.0,1.0,2.0
2,3210110,0.437703,1.0,1.0,3.0
3,3210110,0.773147,1.0,1.0,4.0
4,3210110,0.506952,1.0,1.0,5.0


# New Section

In [ ]:
mydata= pd.read_csv("/content/cell3.csv")
mydata.head()
# Keep only the first 50000 rows
mydata = mydata.head(10000000)
# Extract the dfof and sleep variables
dfof = mydata['dfof'].values
sleep = mydata['sleep_numeric'].values
roi_label = mydata['roi_label'].values
cell_label = mydata['cell_numeric'].values

# Find the unique roi_labels and cell_labels
unique_rois = np.unique(roi_label)
unique_cells = np.unique(cell_label)

# Create an empty dictionary to store the 2D arrays
dfof_by_roi_cell = {}

# Create an empty dictionary to store the info matrix
info_by_roi_cell = {}

# Loop over each unique roi_label and cell_label
for roi in unique_rois:
    for cell in unique_cells:
        
        # Find the indices where the roi_label and cell_label match
        roi_indices = np.where((roi_label == roi) & (cell_label == cell))[0]
        
        if len(roi_indices) > 0:
            # Extract the dfof, sleep, and roi_label values for this roi_label and cell_label
            roi_dfof = dfof[roi_indices]
            roi_sleep = sleep[roi_indices]
            roi_info = np.column_stack((np.repeat(roi, len(roi_indices)), np.repeat(cell, len(roi_indices)), sleep[roi_indices], roi_label[roi_indices]))
            
            # Find the indices where the sleep value changes
            sleep_changes = np.where(np.diff(roi_sleep) != 0)[0] + 1
            
            # Split the dfof array based on the sleep changes
            dfof_periods = np.split(roi_dfof, sleep_changes)
            # Split the dfof_periods into smaller chunks if it is too large
            max_size = 10000
            if len(dfof_periods) > max_size:
                dfof_periods = np.array_split(dfof_periods, len(dfof_periods) // max_size + 1)
                dfof_periods = [period for sublist in dfof_periods for period in sublist]

            # Create a 2D array where each element is an array corresponding to a period between sleep value changes
            dfof_matrix = np.empty((len(dfof_periods),), dtype=object)
            dfof_matrix[:] = dfof_periods
            
            # Store the 2D array and info matrix in the corresponding dictionaries
            dfof_by_roi_cell[(roi, cell)] = dfof_matrix
            info_by_roi_cell[(roi, cell)] = roi_info

# Print the dictionary and info matrix for each roi_label and cell_label
for roi, cell in dfof_by_roi_cell.keys():
    print(f'ROI {roi}, Cell {cell}:')
#    print('Data matrix:')
#    print(dfof_by_roi_cell[(roi, cell)])
    print('Info matrix:')
    print(info_by_roi_cell[(roi, cell)])


流式输出内容被截断，只能显示最后 5000 行内容。
         0.20444189,  0.23985662,  0.27414611,  0.2911628 ,  0.3896045 ,
         0.60401433,  0.06386354,  0.3211686 ,  0.18767783,  0.56401334,
         0.11504708,  0.33778259,  0.03794447,  0.10872383,  0.26486524,
         0.09818602,  0.19686143,  0.32690349,  0.28059908,  0.04174989,
         0.32144722,  0.27272826,  0.41668423,  0.07101871,  0.24641465,
        -0.09113881,  0.12521815,  0.05736109,  0.39562411, -0.00748293,
         0.38347463,  0.22188852,  0.13341716,  0.24447631,  0.2759871 ,
         0.19697636,  0.12838223,  0.41254469,  0.31709944,  0.38563821,
         0.23007561,  0.15849857,  0.1860917 ,  0.16187386,  0.18039357,
         0.19230477,  0.23726744,  0.06374619,  0.34348306,  0.44970853,
         0.17433882,  0.46053761,  0.02209257,  0.19488006, -0.09954733,
         0.42824097,  0.19523936,  0.18318186,  0.18725953,  0.16495377,
         0.18313748,  0.17969414,  0.4937283 ,  0.17349318,  0.13967209,
        -0.10817405,  0.

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



流式输出内容被截断，只能显示最后 5000 行内容。
         5.92923128e-01, -3.47706045e-01,  2.49059100e-01, -3.45170229e-01,
         7.87390877e-02,  1.47925372e-01,  7.10832662e-02,  3.40595525e-01,
        -1.39582693e-01,  4.67062542e-02,  3.74359650e-01,  2.07411930e-01,
         2.71969787e-01,  1.54559854e-01,  2.51122488e-01,  1.29960516e-01,
         4.39107880e-01,  1.99411988e-01,  3.15442385e-02,  8.64287535e-02,
         1.97972938e-01, -5.11323324e-02, -2.61792336e-01,  1.95495626e-01,
        -1.05531226e-01, -4.27158002e-01,  2.72452513e-01,  2.63908613e-01,
         1.34586383e-01,  2.55732658e-01, -3.83600125e-02,  6.36140336e-02,
         2.66402848e-01,  3.63336768e-01,  5.61423008e-01,  7.16338751e-01,
         2.88179528e-01,  2.74024602e-01,  3.83445170e-01, -7.58365429e-02,
         3.93974860e-01,  3.64103942e-01,  5.16979343e-01,  8.83670066e-02,
         1.83066981e-01,  2.54481486e-02,  2.71346339e-01,  1.09317842e-01,
         2.41496462e-01,  2.46936151e-01,  5.25592601e-01,  6

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



流式输出内容被截断，只能显示最后 5000 行内容。
         2.21905326e-01,  4.54278794e-01,  4.15491277e-01,  2.11770004e-01,
         1.42290204e-01,  8.91358120e-02,  9.69568908e-02,  6.35725436e-01,
        -2.41002690e-02,  1.86539255e-01,  2.77702270e-01, -1.83426889e-01,
         2.44792735e-01,  8.39169484e-02,  1.63615206e-01,  2.46092418e-01,
         3.49809457e-01,  9.52668558e-02,  8.09737626e-02,  4.24925837e-01,
         1.32378770e-01, -1.45706469e-01,  3.48719921e-01, -2.55019252e-01,
         5.11120339e-03,  1.68553496e-02,  1.11498043e-01,  1.90946471e-01,
         4.74704633e-02,  2.41044406e-01,  3.65094055e-01,  2.42959291e-01,
         4.75493641e-01,  2.40274248e-01,  1.75613800e-01,  9.93257110e-02,
         2.31519474e-01,  1.87865842e-01,  4.08753623e-02,  2.39893288e-01,
        -4.90410448e-02,  3.86080896e-01,  3.54731920e-01, -1.03118770e-02,
         1.28745878e-02, -1.87073300e-01,  1.75422858e-01,  2.73843718e-02,
         2.34396190e-01,  2.81335210e-01,  1.61044898e-01,  1

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
X = to_time_series_dataset(dfof_matrix)
 

model1 = TimeSeriesKMeans(n_clusters=8, metric="dtw",
                         max_iter=10, random_state=1)
 
model1.fit(X)
print(model1.fit_predict(X))
print(model1.predict(X))

[4 1 4 7 4 0 4 0 4 2 5 6 4 0 2 4 4 4 4 4 4 3 1]
[4 1 4 7 4 0 4 0 4 2 5 6 4 0 2 4 4 4 4 4 4 3 1]
